<a href="https://colab.research.google.com/github/rohanv32/01_Data_Features/blob/main/WeeklyAssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Flight Delay Predictor - Weather and Flight Status Checker**

This Jupyter notebook demonstrates the creation of a simple prototype that helps users check flight statuses and determine if weather conditions might cause delays.

## **Features**
- Retrieves flight status using the **AviationStack API**.
- Fetches weather data for a specified city using the **Visual Crossing Weather API**.
- Analyzes weather conditions such as precipitation probability, wind speed, and general conditions to determine if the weather might impact flight delays.
- Displays the flight status and informs the user if bad weather conditions are likely to cause delays.

## **Prerequisites**
To run this notebook, you need to set up API keys for:
- **AviationStack API** (for retrieving flight status information)
- **Visual Crossing Weather API** (for fetching weather data)


In [ ]:
import requests
from google.colab import userdata

#Here is where the API keys are retrieved.
aviationstack_api_key = userdata.get("AVIATIONSTACK_API_KEY")
visualcrossing_api_key = userdata.get("VISUALCROSSING_API_KEY")

if aviationstack_api_key:
    print("The AviationStack API key has been retrieved successfully!")
else:
    print("Failed to retrieve the AviationStack API key.")

if visualcrossing_api_key:
    print("The VisualStack API key has been retrieved successfully!")
else:
    print("Failed to retrieve the VisualStack API key.")

The AviationStack API key has been retrieved successfully!
The VisualStack API key has been retrieved successfully!


## Bad Weather Detection Function

- The is_bad_weather function determines if the weather is considered adverse based on specified criteria. It evaluates the probability of precipitation, wind speed, and the presence of keywords like 'rain' or 'snow' in the weather conditions. If any of these conditions exceed their respective thresholds, the function returns True, indicating bad weather; otherwise, it returns False.

In [ ]:
def is_bad_weather(precip_prob, wind_speed, conditions):
    # Here, we define thresholds for bad weather based on specific weather conditions.
    if precip_prob > 50 or wind_speed > 20 or 'rain' in conditions.lower() or 'snow' in conditions.lower():
        return True
    return False

- When prompted, enter:
  - A **flight number** (e.g., `DL1087`).

In [ ]:
#Getting the user's input for the flight number.
flight_number = input("Enter the flight number (e.g., EK5): ")

Enter the flight number (e.g., EK5): KL907


## Flight Status and Weather Information Retrieval

- The code constructs a URL to access flight status data from the AviationStack API, using the flight number and API key. It sends a GET request to fetch the flight data and parses the JSON response. If flight data is available, it extracts the flight status, departure city, and arrival city, then prints this information.

- For the departure city, we then fetch current weather details using the fetch_weather function. The code prints weather conditions, precipitation probability, and wind speed. It then checks if the weather is considered bad using the is_bad_weather function and prints a corresponding message if necessary.

- Similarly, for the arrival city, the code fetches weather details, prints the conditions, and checks for bad weather. It also evaluates if the flight status could be impacted by the weather at the arrival city, providing a relevant message if the flight is delayed or on time.

- If no flight data is available, a message indicating this is printed.





In [ ]:
#Fetching the flight's current status from the AviationStack API
flight_url = f"http://api.aviationstack.com/v1/flights?access_key={aviationstack_api_key}&flight_iata={flight_number}"
flight_response = requests.get(flight_url)
flight_data = flight_response.json()

if flight_data['data']:
    flight_info = flight_data['data'][0]

    flight_status = flight_info['flight_status']


    departure_city = flight_info['departure']['airport']
    arrival_city = flight_info['arrival']['airport']

    print(f"\nFlight Status for {flight_number}: {flight_status}")
    print(f"Departure City: {departure_city}")
    print(f"Arrival City: {arrival_city}")

    #This block deals with fetching and displaying weather details for the departure city.
    departure_weather = fetch_weather(departure_city)
    if departure_weather:
        dep_conditions = departure_weather['days'][0]['conditions']
        dep_precip_prob = departure_weather['days'][0]['precipprob']
        dep_wind_speed = departure_weather['days'][0]['windspeed']

        print(f"\nWeather in {departure_city}: {dep_conditions}")
        print(f"Precipitation Probability: {dep_precip_prob}%")
        print(f"Wind Speed: {dep_wind_speed} mph")

        if is_bad_weather(dep_precip_prob, dep_wind_speed, dep_conditions):
            print(f"Bad weather detected in {departure_city}.")

  #Similarly, this block fetches and displays weather details for the arrival city.
    arrival_weather = fetch_weather(arrival_city)
    if arrival_weather:
        arr_conditions = arrival_weather['days'][0]['conditions']
        arr_precip_prob = arrival_weather['days'][0]['precipprob']
        arr_wind_speed = arrival_weather['days'][0]['windspeed']

        print(f"\nWeather in {arrival_city}: {arr_conditions}")
        print(f"Precipitation Probability: {arr_precip_prob}%")
        print(f"Wind Speed: {arr_wind_speed} mph")

        if is_bad_weather(arr_precip_prob, arr_wind_speed, arr_conditions):
            print(f"Bad weather detected in {arrival_city}.")
            if flight_status == 'delayed':
                print("The flight is already delayed, and weather at the arrival city might be a factor.")
            else:
                print("The flight is currently on time, but bad weather at the arrival city might cause a delay.")
else:
    print(f"No flight data available for flight {flight_number}.")


Flight Status for KL907: scheduled
Departure City: Schiphol
Arrival City: Inverness

Weather in Schiphol: Rain, Partially cloudy
Precipitation Probability: 100.0%
Wind Speed: 23.0 mph
Bad weather detected in Schiphol.

Weather in Inverness: Rain, Partially cloudy
Precipitation Probability: 100.0%
Wind Speed: 15.9 mph
Bad weather detected in Inverness.
The flight is currently on time, but bad weather at the arrival city might cause a delay.
